# End-to-End Imaging Demo (DICOM → NIfTI → QC → Histogram)

This notebook demonstrates a minimal, reproducible imaging pipeline:
1. **Load a DICOM series** from a folder
2. **Stack to a 3D volume** and (if possible) convert to HU
3. **Save** as NumPy (`.npy`) and **optionally** NIfTI (`.nii.gz`)
4. **Visual QC panel** with window/level
5. **Simple metrics** (HU histogram, slice statistics)

> Requirements: `pydicom`, `nibabel` (optional for NIfTI), `numpy`, `matplotlib`


In [ ]:
# Configure paths (EDIT THIS CELL)
dicom_dir = r'path/to/your/dicom_series'  # ← set to a folder containing a single CT series
out_np   = '05_python_basics/figures/ct_volume.npy'
out_nii  = '05_python_basics/figures/ct_volume.nii.gz'  # requires nibabel
qc_png   = '05_python_basics/figures/qc_panel_demo.png'
# Window/level for QC (liver-ish example)
wl_center, wl_width = 50, 350


In [ ]:
# Imports
import os, numpy as np, matplotlib.pyplot as plt
import pydicom
try:
    import nibabel as nib
    HAVE_NIB = True
except Exception:
    HAVE_NIB = False
print('Nibabel available:', HAVE_NIB)


In [ ]:
# Helper: sorting key using orientation (preferred) then InstanceNumber
import numpy as _np
def slice_key(ds):
    ipp = getattr(ds, 'ImagePositionPatient', None)
    iop = getattr(ds, 'ImageOrientationPatient', None)
    if ipp is not None and iop is not None and len(iop) >= 6:
        row = _np.array(iop[0:3], float); col = _np.array(iop[3:6], float)
        normal = _np.cross(row, col); pos = _np.array(ipp, float)
        return float(_np.dot(pos, normal))
    try:
        return float(getattr(ds, 'InstanceNumber', 0))
    except Exception:
        return 0.0

def to_hu(arr, ds):
    arr = arr.astype('float32')
    slope = float(getattr(ds, 'RescaleSlope', 1.0))
    inter = float(getattr(ds, 'RescaleIntercept', 0.0))
    return arr * slope + inter


In [ ]:
# 1) Read DICOM series and stack to 3D volume (HxWxZ)
from pathlib import Path
dcm_paths = [p for p in Path(dicom_dir).rglob('*') if p.is_file()]
dsets = []
for p in dcm_paths:
    try:
        ds = pydicom.dcmread(str(p), stop_before_pixels=False, force=True)
        if hasattr(ds, 'SOPInstanceUID'):
            dsets.append(ds)
    except Exception:
        pass
assert dsets, 'No DICOM slices found — check dicom_dir'
dsets.sort(key=slice_key)

vol_list = []
for ds in dsets:
    arr = ds.pixel_array
    arr = to_hu(arr, ds)  # best-effort HU
    vol_list.append(arr)
vol = np.stack(vol_list, axis=-1)
print('Volume shape (HxWxZ):', vol.shape, 'dtype:', vol.dtype)


In [ ]:
# 2) Save NumPy (and NIfTI if available)
os.makedirs(os.path.dirname(out_np), exist_ok=True)
np.save(out_np, vol)
print('Saved .npy to', out_np)
if HAVE_NIB:
    # Construct a simple affine if possible
    try:
        iop = np.array(dsets[0].ImageOrientationPatient, float)
        row, col = iop[0:3], iop[3:6]
        normal = np.cross(row, col)
        ps = np.array(dsets[0].PixelSpacing, float)  # [row, col]
        # Estimate slice spacing from first/last
        if hasattr(dsets[0],'ImagePositionPatient') and hasattr(dsets[-1],'ImagePositionPatient') and vol.shape[-1]>1:
            ipp0 = np.array(dsets[0].ImagePositionPatient, float)
            ipp1 = np.array(dsets[-1].ImagePositionPatient, float)
            dist = np.dot((ipp1-ipp0), normal)
            zspc = abs(dist)/(vol.shape[-1]-1)
        else:
            zspc = float(getattr(dsets[0],'SpacingBetweenSlices', getattr(dsets[0],'SliceThickness',1.0)))
        R = np.column_stack((row*ps[1], col*ps[0], normal*zspc))
        A = np.eye(4); A[:3,:3]=R; A[:3,3]=np.array(getattr(dsets[0],'ImagePositionPatient',[0,0,0]), float)
    except Exception:
        A = np.eye(4)
    nib.save(nib.Nifti1Image(vol, A), out_nii)
    print('Saved NIfTI to', out_nii)


In [ ]:
# 3) Visual QC: window/level grid of slices
def wl_norm(x, c, w):
    low, high = c - w/2, c + w/2
    x = np.clip(x, low, high)
    x = (x - low) / max(w, 1e-6)
    return (x*255).astype(np.uint8)

depth = vol.shape[-1]
rows, cols = 4, 4
idx = np.linspace(0, depth-1, num=rows*cols, dtype=int)
panel = wl_norm(vol, wl_center, wl_width)
plt.figure(figsize=(cols*2.5, rows*2.5))
for i, z in enumerate(idx):
    plt.subplot(rows, cols, i+1)
    plt.imshow(panel[..., z], cmap='gray', vmin=0, vmax=255)
    plt.title(f'z={z}', fontsize=8); plt.axis('off')
plt.tight_layout()
plt.savefig(qc_png, dpi=200)
print('Saved QC panel to', qc_png)


In [ ]:
# 4) Simple metrics: HU histogram (central slice) and summary stats
mid = vol.shape[-1]//2
sl = vol[..., mid]
print('Slice', mid, 'mean=%.1f, sd=%.1f, min=%.1f, max=%.1f' % (sl.mean(), sl.std(), sl.min(), sl.max()))
plt.figure(figsize=(6,4))
plt.hist(sl.ravel(), bins=100, color='steelblue', alpha=0.9)
plt.xlabel('HU'); plt.ylabel('Count'); plt.title('Central slice HU histogram')
plt.tight_layout(); plt.show()
